In [5]:
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 34.1MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 24.2MB/s]                            


In [10]:
import os
def write_lines(path, lines):
    with open(path, 'w', encoding='utf-8') as f :
        for line in lines:
            f.write(f"{line}\n")
write_lines("./ratings_train.txt", nsmc.train.get_all_texts())
write_lines("./ratings_test.txt", nsmc.test.get_all_texts())

In [11]:
from tokenizers import ByteLevelBPETokenizer

BPE_tokenizer=ByteLevelBPETokenizer()
BPE_tokenizer.train(
    files=["./ratings_train.txt","./ratings_test.txt"],
    vocab_size = 10000,
    special_tokens=['PAD']
)

BPE_tokenizer.save_model("./")

['./vocab.json', './merges.txt']

In [13]:
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=["./ratings_train.txt","./ratings_test.txt"],
    vocab_size = 10000
    
)

wordpiece_tokenizer.save_model('./wordpiece')

['./wordpiece/vocab.txt']

In [32]:
from transformers import GPT2Tokenizer
tokenizer_gpt = GPT2Tokenizer.from_pretrained('./bpe')
tokenizer_gpt.pad_token = '[PAD]'

In [40]:
sentences = [
    '아 더빙.. 진짜 짜증나네요 목소리',
    "흠... 포스터보고 초딩여화인줄....오버연기조차 가볍지 않구나",
    "별루 였다..."
]

tokenized_sentences = [tokenizer_gpt.tokenize(sentence) for sentence in sentences]
tokenizer_gpt.add_special_tokens({'pad_token': '[PAD]'})

1

In [41]:
tokenized_sentences

[['ìķĦ', 'ĠëįĶë¹Ļ', '..', 'Ġì§Ħì§ľ', 'Ġì§ľì¦ĿëĤĺ', 'ëĦ¤ìļĶ', 'Ġëª©ìĨĮë¦¬'],
 ['íĿł',
  '...',
  'Ġíı¬ìĬ¤íĦ°',
  'ë³´ê³ł',
  'Ġì´ĪëĶ©',
  'ìĹ¬',
  'íĻĶ',
  'ìĿ¸ì¤Ħ',
  '....',
  'ìĺ¤ë²Ħ',
  'ìĹ°ê¸°',
  'ì¡°ì°¨',
  'Ġê°Ģë³į',
  'ì§Ģ',
  'ĠìķĬ',
  'êµ¬ëĤĺ'],
 ['ë³Ħë£¨', 'Ġìĺ', 'Ģëĭ¤', '...']]

In [42]:
batch_input = tokenizer_gpt(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True
)

In [43]:
batch_input

{'input_ids': [[334, 2338, 263, 581, 4055, 464, 3808, 10001, 10001, 10001, 10001, 10001], [3693, 336, 2720, 758, 2883, 493, 276, 2118, 422, 9875, 875, 2960], [4957, 451, 3653, 336, 10001, 10001, 10001, 10001, 10001, 10001, 10001, 10001]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]}